#Laser calibration

In this article is presented the laser calibration process, that is, the method to determine the position and orientation of the laser, respect to the camera.

##Capture laser point cloud

Pattern is automatically moved on the turntable from -90º to 90º respect to the initial position. Each step (5º) it is captured a point cloud of the laser ([Laser segmentation]()) intersecting with the plane of the pattern ([Point cloud generation]()).

At the end of the capture process, a complete point cloud of the laser is obtained. All points in this point cloud are near to the laser plane. Therefore, this point cloud will be used to determine the plane parameters, as is, the normal vector and the minimum distance respect to the camera coordinate system.

In [1]:
import numpy as np

#Load point cloud
X = np.array([[1,2,3], [3,2,3], [4,1,3], [0,-1,3]])

##Fit laser plane

In order to fit a plane from a point cloud $\textbf{X}$, SVD (Singular Value Decomposition) can be used. This method provides a fast way to obtain the normal $\textbf{n}$ and the distance $d$.

$$
\textbf{X}_i = \begin{bmatrix} x_i \\ y_i \\ z_i \\ \end{bmatrix}
\:\:\:\:\:\:\:\:\:\:
\overline{\textbf{X}} = \frac{1}{n}{\sum{\textbf{X}_i}}
\:\:\:\:\:\:\:\:\:\:
\widetilde{\textbf{X}_i} = \textbf{X}_i - \overline{\textbf{X}}
\:\:\:\:\:\:\:\:\:\:
M = \begin{bmatrix} \widetilde{\textbf{X}_1} \: ... \: \widetilde{\textbf{X}_i} \: ... \: \end{bmatrix}
$$

###Problem
$$
min \: M^T \textbf{n}
\:\:\:\:\:\:
\| \textbf{n} \| = 1
$$
###Solution
$$
M = USV^T
\:\:\:\:\:\:
\textbf{n} = U \begin{bmatrix} 0 \\ 0 \\ 1 \\ \end{bmatrix}
\:\:\:\:\:\:
d = - \textbf{n} \: \overline{\textbf{X}}
$$

In [2]:
from scipy.sparse import linalg

n = X.shape[0]
Xm = X.sum(axis=0) / n
M = np.array(X - Xm).T

U = linalg.svds(M, k=2)[0]

n = np.cross(U.T[0], U.T[1])
if n[2] < 0:
    n *= -1

d = np.dot(n, Xm)

print("\nNormal vector\n\n{0}\n".format(n))
print("\nPlane distance\n\n{0} mm\n".format(d))


Normal vector

[ 0.  0.  1.]


Plane distance

2.999999999999999 mm



##References
* [SVD fitting](http://www.ltu.se/cms_fs/1.51590!/svd-fitting.pdf)